In [1155]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows' , 200)

In [1156]:
company_name = 'mastek'.upper()

In [1157]:
particulars = 'particulars'

In [1158]:
share_price_in = 1000000;

In [1159]:
def calculate_growth(current , previous) :
    if current == 0 and previous == 0 or current == 0 and previous != 0:
        return 0
    if current != 0 and previous == 0 :
        return current / 100
    return ((current/previous) - 1)
       

In [1160]:
def calculate_growth_series(curr_df,prev_df):
    temp_df = curr_df.copy()
    for index in curr_df.index :
        temp_df.loc[index] = calculate_growth(curr_df.loc[index],prev_df[index])
    return temp_df
    

In [1161]:
def get_share_price_in(price) :
    return round((price / share_price_in),2)

    

In [1162]:
def vertical_analysis(item , divider) :
    return item/divider
    

In [1163]:
def round_off_to(num , limit) :
    return round(num,limit)

In [1164]:
def count_leading_space(s): 
    match = re.search(r"^\s*", s) 
    return 0 if not match else match.end()


In [1165]:
def getColumnFormatterbyPercentage(type) :
    columns_perc_formats = {}
    if type == 'income_statement' :
        for column in income_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'balance_sheet' :
        for column in balance_sheet.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'cashflow_statement' :
        for column in cashflow_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    return columns_perc_formats

In [1166]:
def calculate_ratio(dividend , divisor) : 
    return dividend.div(divisor.values).apply(lambda x: round(x,2))

In [1167]:
def styles(color = 'white',backcolor = '#308D46'):
    return [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "150%"),
                              ("color", color),
                              ("background", backcolor),
                              ("font-weight", '600'),
                              ("text-transform", 'uppercase')
                             ])]

In [1168]:
income_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-income.xls')
balance_sheet = pd.read_excel('data/'+company_name+'/'+company_name+'-balance.xls')
cashflow_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-cashflow.xls')

In [1169]:
start_col = income_statement.columns[1];
start_col_balance_sheet = balance_sheet.columns[1];
start_col_cashflow = cashflow_statement.columns[1];

In [1170]:
income_statement_temp_columns = income_statement.columns.values
income_statement_temp_columns[0] = particulars
income_statement.columns = income_statement_temp_columns

balance_sheet_temp_columns = balance_sheet.columns.values
balance_sheet_temp_columns[0] = particulars
balance_sheet.columns = balance_sheet_temp_columns

cashflow_statement_temp_columns = cashflow_statement.columns.values
cashflow_statement_temp_columns[0] = particulars
cashflow_statement.columns = cashflow_statement_temp_columns

In [1171]:
basic_eps_filter = income_statement[particulars].eq('Basic EPS')
basic_eps_pos = income_statement[basic_eps_filter].index[0]

In [1172]:
WASO_filter = income_statement[particulars].isin(['Basic Weighted Average Shares Outstanding','Diluted Weighted Average Shares Outstanding','Basic WASO','Diluted WASO'])
waso_pos = income_statement[WASO_filter].index

In [1173]:
income_statement.dropna(how='all',subset=income_statement.columns[1:], inplace=True)
balance_sheet.dropna(how='all',subset=balance_sheet.columns[1:], inplace=True)
cashflow_statement.dropna(how='all',subset=cashflow_statement.columns[1:], inplace=True)
income_statement.fillna(0 , inplace=True)
balance_sheet.fillna(0 , inplace=True)
cashflow_statement.fillna(0 , inplace=True)

In [1174]:
temp_df = income_statement.loc[0:basic_eps_pos,start_col:].copy()
temp_df = pd.DataFrame(temp_df)


In [1175]:
temp_df = temp_df.apply(get_share_price_in)

In [1176]:
income_statement.loc[0:basic_eps_pos,start_col:] = temp_df

In [1177]:
income_statement.loc[waso_pos,start_col:] = income_statement.loc[waso_pos,start_col:].apply(get_share_price_in)

In [1178]:
temp_revenue = income_statement.loc[1]
income_statement.loc[1] = income_statement.loc[0]
income_statement.loc[0] = temp_revenue

In [1179]:
total_rev = income_statement[particulars].index == 0
income_statement.loc[total_rev,particulars] = income_statement.loc[total_rev,particulars].values[0].strip() 

In [1180]:
is_highlighted_indexes = pd.DataFrame(income_statement.loc[0:basic_eps_pos - 1,particulars])
is_highlighted_indexes = is_highlighted_indexes[particulars].apply(count_leading_space).isin([0,4])
is_highlighted_indexes = is_highlighted_indexes.loc[is_highlighted_indexes].index

In [1181]:
balance_sheet.loc[0:,start_col_balance_sheet:] =  balance_sheet.loc[0:,start_col_balance_sheet:].applymap(get_share_price_in)

In [1182]:
total_equity_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Equity')
total_equity_row = balance_sheet.loc[total_equity_filter,start_col_balance_sheet:].index.values[0]
total_assets_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Assets')
total_assets_row = balance_sheet.loc[total_assets_filter,start_col_balance_sheet:]
total_liabilities_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Liabilities')
total_liabilities_row = balance_sheet.loc[total_liabilities_filter,start_col_balance_sheet:].index.values[0]
balance_sheet.loc[total_liabilities_filter,start_col_balance_sheet:] = balance_sheet.loc[[total_equity_row,total_liabilities_row],start_col_balance_sheet:].sum().values

In [1183]:
cashflow_statement.loc[0:,start_col_cashflow:] =  cashflow_statement.loc[0:,start_col_cashflow:].applymap(get_share_price_in)

In [1184]:
vertical_analysis_is = pd.DataFrame(columns=income_statement.columns) 
vertical_analysis_is[particulars] = income_statement.loc[0:basic_eps_pos - 1,particulars].copy()

In [1185]:
total_revenue_filter = income_statement[particulars].apply(lambda x: x.strip()).eq('Total Revenue')
total_revenue = income_statement.loc[total_revenue_filter,start_col:]

In [1186]:
for column in vertical_analysis_is.columns[1:] : 
    vertical_analysis_is[column] = income_statement.loc[0:basic_eps_pos - 1,column].apply(vertical_analysis,divider=total_revenue[column])
vertical_analysis_is.loc[0,particulars] = vertical_analysis_is.loc[0,particulars].strip() 

In [1187]:
def row_style(row , list , key , color,bgcolor):
   if row[key] in list :
       return pd.Series('color:'+color+';background-color:'+bgcolor, row.index)
   else :
        return pd.Series('', row.index)

In [1188]:
fil = vertical_analysis_is.loc[0:basic_eps_pos - 1,particulars].apply(count_leading_space).eq(0)
templist = vertical_analysis_is.loc[fil,particulars].values
vertical_analysis_is_style = vertical_analysis_is
vertical_analysis_is_style = vertical_analysis_is_style.loc[is_highlighted_indexes].style.apply(row_style,list=templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " income statement vertical analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 
fil = None
templist = None

In [1189]:
balance_sheet_va_results_till = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Non-Controlling/Minority Interests in Equity')
balance_sheet_va_results_till = balance_sheet[balance_sheet_va_results_till].index.values[0]
bs_va_temp = balance_sheet.loc[:balance_sheet_va_results_till]
bs_va_temp = pd.DataFrame(bs_va_temp)
filt = bs_va_temp[particulars].apply(count_leading_space).isin([0,4,8])
balance_sheet_va = pd.DataFrame(bs_va_temp[filt])
bs_va_temp = None
balance_sheet_va_results_till = None
filt = None

In [1190]:
for column in balance_sheet_va.columns[1:] :
    balance_sheet_va[column] = balance_sheet_va.loc[:,column].apply(vertical_analysis,divider=total_assets_row[column])

In [1191]:
bs_fil = balance_sheet_va[particulars].apply(count_leading_space).isin([0,4])
bs_templist = balance_sheet_va.loc[bs_fil,particulars].values
balance_sheet_va_style = balance_sheet_va
balance_sheet_va_style = balance_sheet_va_style.style.apply(row_style,list=bs_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " balance sheet vertical analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('balance_sheet'),precision=2) 
bs_fil = None
bs_templist = None

In [1192]:
#vertical_analysis_is_style

In [1193]:
#balance_sheet_va_style

In [1194]:
income_statement_ha = pd.DataFrame(columns= income_statement.columns)

In [1195]:
income_statement_ha[particulars] = income_statement[particulars]

In [1196]:
income_statement_ha.fillna(0 , inplace=True)

In [1197]:
temp_ha_filt = income_statement[particulars].apply(count_leading_space).isin([0,4])
for index , column in enumerate(income_statement_ha.columns) :
    if index < 2 :
        pass;
    else:
        prevColumn = income_statement.columns[index-1]
        income_statement_ha.loc[temp_ha_filt,column] = calculate_growth_series(income_statement.loc[temp_ha_filt,column],income_statement.loc[temp_ha_filt,prevColumn])
        

In [1198]:
income_statement_ha = income_statement_ha[temp_ha_filt]

In [1199]:
#income_statement_ha.replace([float('inf'), float('-inf')], None, inplace=True)
#income_statement_ha.fillna(0 , inplace=True)

In [1200]:
temp_ha_filt = income_statement[particulars].apply(count_leading_space).isin([0])
iha_templist = income_statement_ha.loc[temp_ha_filt,particulars].values
income_statement_ha_style = income_statement_ha
income_statement_ha_style = income_statement_ha_style.style.apply(row_style,list=iha_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " Income statement horizontal analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 

In [1201]:
balance_sheet_ha_results_till = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Non-Controlling/Minority Interests in Equity')
balance_sheet_ha_results_till = balance_sheet[balance_sheet_ha_results_till].index.values[0]
bs_ha_temp = balance_sheet.loc[:balance_sheet_ha_results_till]
bs_ha_temp = pd.DataFrame(bs_ha_temp)
filt = bs_ha_temp[particulars].apply(count_leading_space).isin([0,4,8])
balance_sheet_ha = pd.DataFrame(bs_ha_temp[filt])

In [1202]:
for index , column in enumerate(balance_sheet_ha.columns) :
    if index < 2 :
        pass;
    else:
        prevColumn = balance_sheet_ha.columns[index-1]
        balance_sheet_ha.loc[:,column] = calculate_growth_series(balance_sheet.loc[:,column],balance_sheet.loc[:,prevColumn])
balance_sheet_ha[start_col_balance_sheet] = 0.0
prevColumn = None

In [1203]:
bs_fil_ha = balance_sheet_ha[particulars].apply(count_leading_space).isin([0,4])
bs_templist_ha = balance_sheet_ha.loc[bs_fil_ha,particulars].values
balance_sheet_ha_style = balance_sheet_ha
balance_sheet_ha_style = balance_sheet_ha_style.style.apply(row_style,list=bs_templist_ha,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " balance sheet horizontal analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('balance_sheet'),precision=2) 
bs_fil_ha = None
bs_templist_ha = None

In [1204]:
cashflow_statement_filt = cashflow_statement[particulars].apply(count_leading_space).isin([0,8,12])
cash_flow_stmt_ha = pd.DataFrame(cashflow_statement[cashflow_statement_filt])
cash_flow_stmt_ha.reset_index(inplace=True, drop=True)
cash_flow_stmt_ha_temp = cash_flow_stmt_ha.copy()

In [1205]:
for index , column in enumerate(cash_flow_stmt_ha.columns) :
    if index < 2 :
        pass;
    else:
        prevColumn = cash_flow_stmt_ha.columns[index-1]
        cash_flow_stmt_ha.loc[:,column] = calculate_growth_series(cash_flow_stmt_ha_temp.loc[:,column],cash_flow_stmt_ha_temp.loc[:,prevColumn])
cash_flow_stmt_ha[start_col_cashflow] = 0.0
cash_flow_stmt_ha_temp = None
cashflow_statement_filt = None
prevColumn = None

In [1206]:
cf_fil_ha = cash_flow_stmt_ha[particulars].apply(count_leading_space).isin([0])
cf_templist_ha = cash_flow_stmt_ha.loc[cf_fil_ha,particulars].values
cash_flow_stmt_ha_style = cash_flow_stmt_ha
cash_flow_stmt_ha_style = cash_flow_stmt_ha_style.style.apply(row_style,list=cf_templist_ha,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " cash flow horizontal analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('cashflow_statement'),precision=2) 
cf_fil_ha = None
cf_templist_ha = None

In [1207]:
#income_statement_ha_style

In [1208]:
#balance_sheet_ha_style

In [1209]:
#cash_flow_stmt_ha_style

In [1210]:
is_trend_base = income_statement[start_col]

In [1211]:
income_statement_trend = pd.DataFrame(columns=income_statement.columns)
income_statement_trend[particulars] = income_statement[particulars]
income_statement_trend[start_col] = 0
income_statement_trend.fillna(0,inplace=True)

In [1212]:
for index , column in enumerate(income_statement_trend.columns) :
    if index < 2 :
        pass;
    else:
        income_statement_trend.loc[:,column] = calculate_growth_series(income_statement.loc[:,column],is_trend_base)
is_trend_base = None      

In [1213]:
temp_ta_filt = income_statement_trend[particulars].apply(count_leading_space).isin([0])
ita_templist = income_statement_trend.loc[temp_ta_filt,particulars].values
income_statement_trend_style = income_statement_trend
income_statement_trend_style = income_statement_trend_style.style.apply(row_style,list=ita_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " Income statement trend analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 

In [1214]:
temp_balance = balance_sheet.loc[:balance_sheet_ha_results_till]

In [1215]:
bs_trend_base = temp_balance[start_col_balance_sheet]
balance_sheet_trend = pd.DataFrame(columns=temp_balance.columns)
balance_sheet_trend[particulars] = temp_balance[particulars]
balance_sheet_trend[start_col] = 0
balance_sheet_trend.fillna(0,inplace=True)

In [1216]:
for index , column in enumerate(balance_sheet_trend.columns) :
    if index < 2 :
        pass;
    else:
        balance_sheet_trend.loc[:,column] = calculate_growth_series(temp_balance.loc[:,column],bs_trend_base)
temp_balance = None
bs_trend_base = None

In [1217]:
bs_fil_ta = balance_sheet_trend[particulars].apply(count_leading_space).isin([0,4])
bs_templist_ta = balance_sheet_trend.loc[bs_fil_ta,particulars].values
balance_sheet_trend_style = balance_sheet_trend
balance_sheet_trend_style = balance_sheet_trend_style.style.apply(row_style,list=bs_templist_ta,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " balance sheet trend analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('balance_sheet'),precision=2) 
bs_fil_ta = None
bs_templist_ta = None

In [1218]:
cashflow_statement_filt = cashflow_statement[particulars].apply(count_leading_space).isin([0,8,12])
cash_flow_stmt_trend = pd.DataFrame(columns=cashflow_statement.columns)
cash_flow_stmt_trend_temp = cashflow_statement[cashflow_statement_filt].copy()
cash_flow_stmt_trend_temp.reset_index(inplace=True,drop=True)
cash_flow_stmt_trend[particulars] = cash_flow_stmt_trend_temp[particulars]
cash_flow_stmt_trend[start_col_cashflow] = 0

In [1219]:
cf_trend_base = cash_flow_stmt_trend_temp[start_col_cashflow]

In [1220]:
for index , column in enumerate(cash_flow_stmt_trend.columns) :
    if index < 2 :
        pass;
    else:
        cash_flow_stmt_trend.loc[:,column] = calculate_growth_series(cash_flow_stmt_trend_temp.loc[:,column],cf_trend_base)
cash_flow_stmt_trend_temp = None
cf_trend_base = None

In [1221]:
cf_fil_ta = cash_flow_stmt_trend[particulars].apply(count_leading_space).isin([0])
cf_templist_ta = cash_flow_stmt_trend.loc[cf_fil_ta,particulars].values
cash_flow_stmt_trend_style = cash_flow_stmt_trend
cash_flow_stmt_trend_style = cash_flow_stmt_trend.style.apply(row_style,list=cf_templist_ta,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " cash flow trend analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('cashflow_statement'),precision=2) 
cf_fil_ta = None
cf_templist_ta = None

In [1222]:
#income_statement_trend_style

In [1223]:
#balance_sheet_trend_style

In [1224]:
#cash_flow_stmt_trend_style

In [1225]:
current_assets_filter = balance_sheet[particulars].apply(lambda x: x.strip().lower()).eq('total current assets')
current_liabilities_filter = balance_sheet[particulars].apply(lambda x: x.strip().lower()).eq('total current liabilities')

In [1226]:
current_assets = balance_sheet.loc[current_assets_filter,start_col_balance_sheet:].copy()
current_liabilities = balance_sheet.loc[current_liabilities_filter,start_col_balance_sheet:].copy()

In [1227]:
solvency_ratio = pd.DataFrame(columns=balance_sheet.columns)

In [1228]:
solvency_ratio[particulars] = ['current ratio' , 'quick ratio', 'cash ratio']

In [1229]:
solvency_ratio.loc[0:0,start_col_balance_sheet:] = calculate_ratio(current_assets,current_liabilities).values

In [1230]:
inventories_filter = balance_sheet[particulars].apply(lambda x: x.strip().lower()).eq('inventories')

In [1231]:
inventories = balance_sheet.loc[inventories_filter,start_col_balance_sheet:].copy()
if len(inventories.value_counts().values) == 0 :
    data = [[0] * len(inventories.columns)]
    inventories = pd.DataFrame(data=data , columns=inventories.columns).copy()
quick_assets = current_assets - inventories.values

In [1232]:
solvency_ratio.loc[1:1,start_col_balance_sheet:] =  calculate_ratio(quick_assets,current_liabilities).values

In [1233]:
cash_and_ceq_filter = balance_sheet[particulars].apply(lambda x: x.strip().lower()).eq('cash and cash equivalents')

In [1234]:
cash_and_ceq = balance_sheet.loc[cash_and_ceq_filter,start_col_balance_sheet:].copy()

In [1235]:
solvency_ratio.loc[2:2,start_col_balance_sheet:] = calculate_ratio(cash_and_ceq,current_liabilities).values

In [1236]:
solvency_ratio

,particulars,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,current ratio,2.42,2.5,2.7,3.02,2.26,2.87,1.85,1.87,1.78
1,quick ratio,2.42,2.5,2.7,3.02,2.26,2.87,1.85,1.87,1.78
2,cash ratio,0.53,0.86,0.86,0.46,0.46,0.55,0.47,0.81,0.85
